**Microsoft Fabric Hackathon Submission**

**Description**: The purpose of this notebook is to create a logistic regression model to solve for the real world use case of looking at the drivers of attendance for nonprofit programming. This notebook relies on standard python packages and outputs a dataframe to the lakehouse that can be consumed by Power BI in a report. 

**Authors**: Shaheen Eshghipour, Maxwell Titsworth

**Date**: 03/01/2024


**Import packages**

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
from patsy import dmatrices
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DoubleType

StatementMeta(, , , Waiting, )

**Import Data**

The below cell generates random dataset that aligns with the schema we created. You can import your own dataset using the options provided in the other cells. 

In [ ]:
# Set a seed for reproducibility
np.random.seed(42)

# Number of rows
num_rows = 20000

# Generate random data for each variable
data = {
    'Age': np.random.randint(18, 100, num_rows),
    'Attended_in_last_30_days': np.random.choice([0, 1], num_rows),
    'Avg_lbs_per_UOS': np.random.uniform(20, 30, num_rows),
    'days_since_enrollment': np.random.randint(0, 1501, num_rows),
    'distance_miles': np.random.uniform(0, 50, num_rows),
    'Gender': np.random.choice(["Male", "Female", "Other"], num_rows),
    'HHSize': np.random.choice(["1-10"], num_rows),
    'Menu_Tier': np.random.choice(["Tier A", "Tier B"], num_rows),
    'Primary_Distribution_Day': np.random.choice(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"], num_rows),
    'Program_Name': np.random.choice(["Pop-up", "Neighborhood Pantry", "Brownbag"], num_rows),
    'StartTimeBin': np.random.choice(["Morning", "Early Afternoon", "Late Afternoon"], num_rows),
    'Race': np.random.choice(["Black", "White", "Asian"], num_rows)  # Replace with actual race categories
}

# Create DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df.head())

#importing the dateframe from a .csv
#df = pd.read_csv("~/participant_data.csv")


**Split into train/test sets**
<br>Setting random_state = 1 ensures the split is the same every time this code is run
<br>A 70,30 train/test split is used

In [ ]:
train, test = train_test_split(df, test_size=0.3, random_state=1)

StatementMeta(, , , Waiting, )

**Create Logistic Regression model**

In [ ]:
model = smf.glm(formula="Attended_in_last_30_days ~ Age + distance_miles + days_since_enrollment+ C(Gender) + HHSize + C(Menu_Tier) + C(Primary_Distribution_Day) + C(Program_Name) + C(Race) + C(StartTimeBin)", data=train, family=sm.families.Binomial()).fit()
print(model.summary())

In [ ]:
model.params # get log odds
model.conf_int(alpha=0.05, cols=None) # get log CI's

np.exp(model.params) # get odds
np.exp(model.conf_int(alpha=0.05, cols=None)) # get CI's

model.summary()

model.pvalues

**Create Confusion Matrix**

In [ ]:
probs = model.predict(test).values
y_hat = model.predict(test).values

y = test['Attended_in_last_30_days'].values

# define threshold and predict
threshold = 0.5
y_hat = np.where(y_hat >= threshold, 1, 0)

# create confusion matrix
cm = metrics.confusion_matrix(y, y_hat)

# Plot as heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Oranges', fmt='g', linewidths=1, linecolor='black', cbar=False)
plt.xlabel('Y hat')
plt.ylabel('Y')
plt.title('Confusion Matrix')
plt.show()

**Calculate Model Metrics**

In [ ]:
TN = cm[0, 0]  # True negatives


StatementMeta(, , , Waiting, )

**Calculate model metrics**

In [ ]:
# calculate model metrics
TN = cm[0, 0]  # True negatives
FP = cm[0, 1]  # False positives
TP = cm[1, 1]  # True positives
FN = cm[1, 0]  # False negatives

Sensitivity = TP / (TP+FN)
Specificity = TN / (TN+FP)

# show metrics
print("Accuracy =",round(metrics.accuracy_score(y, y_hat),2))
print("Sensitivity = ",round(Sensitivity,2))
print("Specificity = ",round(Specificity,2))

**Extract Coefficients and Load to Table**

In [ ]:
# extract log odds + CI's
log_odds = model.params.values # get log odds
log_CI = model.conf_int(alpha=0.05, cols=None).values # get log CI's

# extract odds + CI's
odds = np.exp(model.params.values) # get odds
CI = np.exp(model.conf_int(alpha=0.05, cols=None).values) # get CI's

# extract pvals
pvals = model.pvalues.values

# extract coefficient names
coeff_names = model.params.index.values

# create data frame
results_df = pd.DataFrame({
    'Name': coeff_names,
    'LogOddsRatio': log_odds,
    'LogOddsLower_Bound_CI': log_CI[:, 0],
    'LogOddsUpper_Bound_CI': log_CI[:, 1],
    'OddsRatio': odds,
    'OddsLower_Bound_CI': CI[:, 0],
    'OddsUpper_Bound_CI': CI[:, 1],
    'pvals': pvals
})

StatementMeta(, , , Waiting, )

**Write to lakehouse**

In [ ]:
# convert data types from object to appropriate type
results_df['Name'] = results_df['Name'].astype('|S')

# define the schema to match DataFrame
schema = StructType([
    StructField("Name", StringType(), True),
    StructField("LogOddsRatio", DoubleType(), True),
    StructField("LogOddsLower_Bound_CI", DoubleType(), True),
    StructField("LogOddsUpper_Bound_CI", DoubleType(), True),
    StructField("OddsRatio", DoubleType(), True),
    StructField("OddsLower_Bound_CI", DoubleType(), True),
    StructField("OddsUpper_Bound_CI", DoubleType(), True),
    StructField("pvals", DoubleType(), True)
    ])

# create a Spark DataFrame using the specified schema
sparkDF = spark.createDataFrame(results_df, schema=schema)

# - write the Spark DataFrame to a table for the first time
#sparkDF.write.format("delta").saveAsTable("attendancemodelcoeffients_fabricsubmission")

# - write the Spark DataFrame to the table subsequent times
sparkDF.write.format("delta").option("overwriteSchema", "true").mode("overwrite").saveAsTable("attendancemodelcoeffients_fabricsubmission")